In [2]:
import pandas as pd
import numpy as np
import keras

In [48]:
import pandas as pd
from pymongo import MongoClient


def _connect_mongo(host, port, db):
    """ A util for making a connection to mongo """
    conn = MongoClient(host, port)
    return conn[db]


def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

df = read_mongo(db="WorldCup", collection="Tweets", no_id=True)
df.head()

,created_at,favorite_count,location,retweeted,retweeted_count,text
0,2018-05-26 13:20:00,0,None,False,0,RT @LFC: When you turn around and spot @Alex_O...
1,2018-05-26 13:20:01,0,JHB|FREE STATE|NORTHERN CAPE|NORTH WEST,False,0,Real Madrid or Liverpool? 🏆\nRonaldo or M.Sala...
2,2018-05-26 13:20:00,0,UK & Ireland,False,0,Massive #UCLfinal offer! ⚽️\n\nBack #RMFC at 4...
3,2018-05-26 13:20:02,0,Bolton,False,0,RT @paddypower: Ahead of the Champions League ...
4,2018-05-26 13:20:02,0,None,False,0,RT @ninanaka: Hope it will be an unforgettable...


In [71]:
from collections import Counter
import re

def flat_list(l):
    list_final = []
    for i in l:
        if type(i) is list:
            list_final.extend(i)
        else:
            list_final.append(i)
    return list_final

def TM_SC_create_words(list_text, nb_min_word = 2):
    list_list_words = [re.findall(r"\w+",x) for x in list_text]
    WORDS = Counter(flat_list(list_list_words))
    WORDS_to_correct = {k:v for (k,v) in WORDS.items() if v <nb_min_word} #Dictionnaire ajouter key value
    WORDS = {k:v for (k,v) in WORDS.items() if v >=nb_min_word}
    return WORDS, WORDS_to_correct

WORDS, WORDS_to_correct = TM_SC_create_words(df["text"].values)

In [50]:
s = 0
for v in WORDS.values(): s += v
print("Nombre de mots:", s)

Nombre de mots: 7777922


In [51]:
df.drop_duplicates(subset="text", keep='first', inplace=True)
df.shape

(184310, 6)

In [52]:
WORDS, WORDS_to_correct = TM_SC_create_words(df["text"].values)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [53]:
s = 0
for v in WORDS.values(): s += v
print("Nombre de mots après drop duplicates text:", s)

Nombre de mots après drop duplicates text: 2291445


In [54]:
def remove_rt(text):
    return re.sub(r"^RT \@(.*)\:", "", text) 

df["text_without_rt"] = df["text"].apply(lambda x: remove_rt(x))

In [56]:
WORDS, WORDS_to_correct = TM_SC_create_words(df["text_without_rt"].values)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
s = 0
for v in WORDS.values(): s += v
print("Nombre de mots après drop duplicates text:", s)

Nombre de mots après drop duplicates text: 2173492


In [60]:
df[["text_without_rt"]].head(100)

,text_without_rt
0,When you turn around and spot @Alex_OxChambo ...
1,Real Madrid or Liverpool? 🏆\nRonaldo or M.Sala...
2,Massive #UCLfinal offer! ⚽️\n\nBack #RMFC at 4...
3,//t.co/cAdY1Ive…
4,Hope it will be an unforgettable journey for ...
5,"""Pick any continent, any country, any city an..."
6,THE BRU GOT THRU TO KIEV 💪\n\nThis should do ...
7,You'll never walk alone bebeim @LFC #UCLFinal
8,Tonight the world is supposed to have a new ba...
9,//t.co/zHveigOdHW


In [67]:
%%time
HASHTAG = "#UCLfinal"
def clean_tweets(text):
    text_clean = re.sub(r"^RT \@(.*)\:", "", text)
    text_clean = re.sub(HASHTAG, "", text_clean)
    text_clean = text_clean.lower()
    text_clean = re.sub(r"(?m)^\s+", "", text_clean)
    text_clean = re.sub("\n", " ", text_clean)
    text_clean = re.sub(r"""(?:(?:https?|ftp):\/\/|\b(?:[a-z\d]+\.))(?:(?:[^\s()<>]+|\((?:[^\s()<>]+|(?:\([^\s()<>]+\)))?\))+(?:\((?:[^\s()<>]+|(?:\(?:[^\s()<>]+\)))?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))?""", "", text_clean)
    text_clean = re.sub("//", "", text_clean)
    return text_clean

df["tweets_cleaned"] = df["text_without_rt"].apply(lambda x: clean_tweets(x))

CPU times: user 17min 47s, sys: 2.19 s, total: 17min 49s
Wall time: 17min 38s


In [73]:
WORDS, WORDS_to_correct = TM_SC_create_words(df["tweets_cleaned"].values)
s = 0
for v in WORDS.values(): s += v
print("Nombre de mots après drop duplicates text:", s)

Nombre de mots après drop duplicates text: 1757370


In [75]:
df[["tweets_cleaned"]].head(100)

,tweets_cleaned
0,when you turn around and spot @alex_oxchambo a...
1,real madrid or liverpool? 🏆 ronaldo or #uclfi...
2,massive offer! ⚽️ back #rmfc at 40/1 or #lfc ...
3,
4,hope it will be an unforgettable journey for t...
5,"""pick any continent, any country, any city and..."
6,the bru got thru to kiev 💪 this should do you ...
7,you'll never walk alone bebeim @lfc #uclfinal
8,tonight the world is supposed to have a new ba...
9,


In [78]:
df_not_empty = df[df["tweets_cleaned"] != ""].copy()
df_not_empty.shape

(176952, 8)

In [79]:
df_not_empty["len_tweet_cleaned"] = df_not_empty["tweets_cleaned"].apply(lambda x: len(x.split()))
df_not_empty.head()

,created_at,favorite_count,location,retweeted,retweeted_count,text,text_without_rt,tweets_cleaned,len_tweet_cleaned
0,2018-05-26 13:20:00,0,None,False,0,RT @LFC: When you turn around and spot @Alex_O...,When you turn around and spot @Alex_OxChambo ...,when you turn around and spot @alex_oxchambo a...,12
1,2018-05-26 13:20:01,0,JHB|FREE STATE|NORTHERN CAPE|NORTH WEST,False,0,Real Madrid or Liverpool? 🏆\nRonaldo or M.Sala...,Real Madrid or Liverpool? 🏆\nRonaldo or M.Sala...,real madrid or liverpool? 🏆 ronaldo or #uclfi...,10
2,2018-05-26 13:20:00,0,UK & Ireland,False,0,Massive #UCLfinal offer! ⚽️\n\nBack #RMFC at 4...,Massive #UCLfinal offer! ⚽️\n\nBack #RMFC at 4...,massive offer! ⚽️ back #rmfc at 40/1 or #lfc ...,14
4,2018-05-26 13:20:02,0,None,False,0,RT @ninanaka: Hope it will be an unforgettable...,Hope it will be an unforgettable journey for ...,hope it will be an unforgettable journey for t...,16
5,2018-05-26 13:20:02,0,"Verulam, South Africa",False,0,"RT @MelissaReddy_: ""Pick any continent, any co...","""Pick any continent, any country, any city an...","""pick any continent, any country, any city and...",20


In [80]:
df_valid = df_not_empty[df_not_empty["len_tweet_cleaned"] > 3]
df_valid.shape

(155700, 9)